# Get Prices from CoinGecko

In [11]:
# price_fetcher.py
import requests

def get_prices():
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        'ids': 'bitcoin,ethereum',
        'vs_currencies': 'usd'
    }
    response = requests.get(url, params=params)
    data = response.json()
    return {coin: data[coin]['usd'] for coin in data}

In [5]:
token_path = r'//project/Trading bot with Telegram integration/Telegram_token.txt'
def load_token(path=token_path):
    with open(path, "r") as f:
        return f.read().strip()


In [6]:
TELEGRAM_TOKEN = load_token()

In [8]:
chat_path = r'//project/Trading bot with Telegram integration/Telegram_chat.txt'
def load_token(path=chat_path):
    with open(path, "r") as f:
        return f.read().strip()

In [10]:
TELEGRAM_CHAT_ID = load_token()

# Rebalancing logic

In [12]:
# rebalance_logic.py
# from config import TARGET_ALLOCATION, THRESHOLD

# portfolio etc could have been set in a config but will give an example target here:
PORTFOLIO = {
    'bitcoin': 0.15,   # in BTC
    'ethereum': 2.0    # in ETH
}

TARGET_ALLOCATION = {
    'bitcoin': 0.6,
    'ethereum': 0.4
}

THRESHOLD = 0.05  # 5% deviation


def get_portfolio_allocation(portfolio, prices):
    values = {coin: amount * prices[coin] for coin, amount in portfolio.items()}
    total = sum(values.values())
    return {coin: val / total for coin, val in values.items()}, values

def needs_rebalancing(current_alloc):
    actions = []
    for coin, target_pct in TARGET_ALLOCATION.items():
        current_pct = current_alloc.get(coin, 0.0)
        deviation = current_pct - target_pct
        if abs(deviation) > THRESHOLD:
            actions.append({
                'asset': coin,
                'action': 'SELL' if deviation > 0 else 'BUY',
                'deviation': deviation
            })
    return actions

# Telegram Notifier

In [13]:
# notifier.py
import requests

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    data = {
        'chat_id': TELEGRAM_CHAT_ID,
        'text': message
    }
    requests.post(url, data=data)

In [14]:
send_telegram_message("Pargooooo")

# Main Orchestrator
Sends message to chat_id

In [17]:
def run_bot():
    prices = get_prices()
    current_alloc, values = get_portfolio_allocation(PORTFOLIO, prices)

    actions = needs_rebalancing(current_alloc)
    if not actions:
        send_telegram_message("Portfolio is balanced. No rebalancing needed.")
        return

    message = "Rebalancing Alert:\n\n"
    for action in actions:
        coin = action['asset']
        pct = current_alloc.get(coin, 0)
        target = TARGET_ALLOCATION[coin]
        message += f"{action['action']} {coin.upper()} | Current: {pct:.2%}, Target: {target:.2%}\n"
    
    send_telegram_message(message)

if __name__ == "__main__":
    run_bot()

# Run It on a Schedule

In [19]:
# schedule_runner.py
import schedule
import time

schedule.every().hour.do(run_bot)

while True:
    schedule.run_pending()
    time.sleep(30)

KeyboardInterrupt: 